In [1]:
import cv2
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from dataset import YooxDatasetHistogram, get_norm_hist
from model_encoder import ColorAutoencoder

from utils import save_checkpoint, load_checkpoint, save_some_examples
import config

from tqdm import tqdm

c:\Users\Mirco\Desktop\CV_pw\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd ..

c:\Users\Mirco\Desktop\CV_pw


In [3]:
from webbrowser import get

train_dataset = YooxDatasetHistogram(True)
train_dataset[0][2]

tensor([[[205., 205., 205.],
         [205., 205., 205.],
         [205., 205., 205.],
         ...,
         [215., 215., 215.],
         [214., 214., 214.],
         [214., 214., 214.]],

        [[205., 205., 205.],
         [205., 205., 205.],
         [205., 205., 205.],
         ...,
         [215., 215., 215.],
         [215., 215., 215.],
         [214., 214., 214.]],

        [[205., 205., 205.],
         [205., 205., 205.],
         [205., 205., 205.],
         ...,
         [215., 215., 215.],
         [214., 214., 214.],
         [214., 214., 214.]],

        ...,

        [[227., 227., 227.],
         [227., 227., 227.],
         [227., 227., 227.],
         ...,
         [235., 235., 235.],
         [235., 235., 235.],
         [234., 234., 234.]],

        [[227., 227., 227.],
         [227., 227., 227.],
         [227., 227., 227.],
         ...,
         [235., 235., 235.],
         [235., 235., 235.],
         [235., 235., 235.]],

        [[227., 227., 227.],
       

In [4]:
from sklearn.cluster import KMeans, SpectralClustering
import matplotlib

In [5]:
def palette(img):
    arr = np.asarray(img)
    palette, index = np.unique(asvoid(arr).ravel(), return_inverse=True)
    palette = palette.view(arr.dtype).reshape(-1, arr.shape[-1])
    count = np.bincount(index)
    order = np.argsort(count)
    return palette[order[::-1]]

def asvoid(arr):
    arr = np.ascontiguousarray(arr)
    return arr.view(np.dtype((np.void, arr.dtype.itemsize * arr.shape[-1])))

def hsv(arr, j):
    return matplotlib.colors.rgb_to_hsv(arr)[j]

def pantone(img):
    C = palette(img)#[:50]
    P = None
    n = 0
    for c in C:
        if P is None:
            P = np.array(c[None,:])
        else:
            for d in P:
                if ((c-d)**2).sum() > 50:
                    P = np.concatenate([P,c[None,:]],axis=0)
                    n += 1
                    if n > 50:
                        break

    C = P
    N_COLS = 5
    model = KMeans(N_COLS)
    model.fit(C)
    C = model.cluster_centers_.astype(np.uint8)
    P = np.zeros((200,N_COLS*100,3)).astype(np.uint8)
    #for j in range(3):
    j = 2
    for c in range(N_COLS):
        for d in range(N_COLS):
            #if hsv(C[c], j) > hsv(C[d], j):
            if hsv(C[c], j) > hsv(C[d], j):
                tmp = C[c].copy()
                C[c] = C[d].copy()
                C[d] = tmp.copy()

    for i, c in enumerate(C):
        #P[100*j:100*(1+j),100*i:100*(1+i),:] = c
        P[:,100*i:100*(1+i),:] = c
    return P

In [6]:
img = cv2.imread("dataset/upper_body/images/denim_denim-shirts/12141279ui_0_r.jpg")
img = cv2.imread("dataset/upper_body/images/knitwear_track-tops/46614975wd_0_d.jpg")
img = cv2.imread("dataset/upper_body/images/knitwear_track-tops/46662850ev_0_d.jpg")
C = pantone(img)
cv2.imshow("1",C)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
enc = ColorAutoencoder(in_channels=3, levels=8).to(config.DEVICE)
opt_gen = optim.Adam(enc.parameters(), lr=config.GEN_LEARNING_RATE, betas=(0.5, 0.999))
load_checkpoint(config.CHECKPOINT_GEN, enc, opt_gen, config.GEN_LEARNING_RATE)

x = torch.tensor(img.transpose(2,0,1)).float()
x = get_norm_hist(x)
enc = enc.eval()
print(x.shape)
enc(x.unsqueeze(0), vec_space=True)

=> Loading checkpoint
torch.Size([3, 256])


tensor([[1.9364, 0.6042, 0.3480, 1.3593]], grad_fn=<AddmmBackward0>)

In [8]:
train_loader = DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=False,
        # num_workers=config.NUM_WORKERS,
    )

In [9]:
all_vec = None
enc.eval()
i = 0
for x, y, z in tqdm(train_loader):
    res = enc(x, True)
    if all_vec is None:
        all_vec = res
    else:
        all_vec = torch.concat([all_vec, res], 0)
        i += 1
        if i > 50:
            pass#break


  4%|▍         | 64/1520 [00:13<04:57,  4.90it/s]


KeyboardInterrupt: 

In [ ]:
campione = 8754

In [ ]:
diff = all_vec[None,:,:] - all_vec[campione,None,:]
diff = diff**2
diff = diff.sum(axis=2)

In [ ]:
for j in range(10):
    i = diff.flatten().argsort()[j+1].item()
    p1 = pantone(train_dataset[campione][2])
    p2 = pantone(train_dataset[i][2])

    cv2.imshow("1",p1)
    cv2.imshow("2",p2)
    cv2.imshow("1v",train_dataset[campione][2].numpy()/255)
    cv2.imshow("2v",train_dataset[i][2].numpy()/255)
    cv2.waitKey()
    cv2.destroyAllWindows()